In [18]:
import numpy as np


x = np.reshape(np.random.random(size=10), (1,-1))
y = np.reshape(np.random.random(size=10), (-1,1))

float(x@y), x@y

/tmp/ipykernel_95078/659430835.py:7: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  float(x@y), x@y


(2.561555106557572, array([[2.56155511]]))

In [1]:
# from prompting.tasks.date_qa import DateQuestionAnsweringTask
from prompting.tasks.task import BaseTask, BaseRewardModel
from prompting.tasks.summarization import SummarizationTask, SummarizationRewardConfig
from prompting.tasks.qa import QuestionAnsweringTask, QARewardConfig

from prompting.tools.datasets.wiki import WikiDataset
from prompting.tools.datasets.base import BaseDataset
from pydantic import BaseModel, ConfigDict
import random
from typing import ClassVar
import bittensor as bt


class TaskConfig(BaseModel):
    task: BaseTask.__class__
    probability: float
    datasets: list[BaseDataset.__class__]
    reward_model: BaseRewardModel.__class__

    model_config = ConfigDict(arbitrary_types_allowed=True)


class TaskRegistry(BaseModel):
    tasks: ClassVar[list[TaskConfig]] = [
        TaskConfig(task=QuestionAnsweringTask, probability=0.6, datasets=[WikiDataset], reward_model=QARewardConfig),
        TaskConfig(
            task=SummarizationTask, probability=0.4, datasets=[WikiDataset], reward_model=SummarizationRewardConfig
        ),
        # TaskConfig(task=DateQuestionAnsweringTask, probability=0.2, datasets=[WikiDateDataset])
    ]

    @classmethod
    def random(cls) -> TaskConfig:
        probabilities = [task.probability for task in cls.tasks]
        selected_task = random.choices(cls.tasks, probabilities)[0]
        return selected_task

    @classmethod
    def get_task_datasets(cls, task: BaseTask.__class__):
        try:
            return [t.datasets for t in cls.tasks if task is t.task][0]
        except Exception:
            bt.logging.error("Tried accessing non-registered task")
            return []

    @classmethod
    def get_random_task_dataset(cls, task: BaseTask.__class__) -> BaseDataset.__class__:
        return random.choice(cls.get_task_datasets(task))

    @classmethod
    def get_task_reward(cls, task: BaseTask) -> BaseRewardModel:
        try:
            return [t.reward_model for t in cls.tasks if task is t.task][0]
        except Exception:
            bt.logging.error("Tried accessing non-registered task")
            return []

    @classmethod
    def create_random_task(cls, llm_pipeline) -> BaseTask:
        task_config = cls.random()
        dataset = cls.get_random_task_dataset(task_config.task)
        return task_config.task(
            llm_pipeline=llm_pipeline, context=dataset().next(), reward_config=task_config.reward_model()
        )


/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-31 16:23:55,367	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:201: UserWarning: Field name "required_hash_fields" in "StreamPromptingSynapse" shadows an attribute in parent "StreamingSynapse"
  warnings.warn(
/root/.cache/pypoetry/virtualenvs/prompting-fb5sw-i7-py3.10/lib/python3.10/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config[

In [5]:
TaskRegistry().get_task_reward(QuestionAnsweringTask)

prompting.tasks.qa.QARewardConfig